<a href="https://colab.research.google.com/github/TheOneTrueGuy/tessellator/blob/main/Tesselator_with_Groq_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq



In [ ]:
import os
# performing a chat completion
from groq import Groq
from google.colab import userdata
# this assumes you have set your Groq API key as a Secret under the key icon
GROQ_API_KEY= userdata.get('GROQ_API_KEY')
client = Groq(
    api_key=GROQ_API_KEY #os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of low latency LLMs",
        }
    ],
    model="mixtral-8x7b-32768",
)
#just a test for now
print(chat_completion.choices[0].message.content)


In [ ]:
import collections
from collections import deque
import networkx as nx
from groq import Groq
inport os

concepts = set()  # Set to store unique concepts encountered

# Function to query the LLM for related concepts (using OpenAI API)
#openai.api_key = "none"
#openai.api_base = "http://localhost:1234/v1"
#client = openai(base_url="http://localhost:1234/v1", api_key="not-needed")
def prompt_lm(prompt):
    print("inside")
    completion =     response = openai.ChatCompletion.create(model="local_model",
      messages=[
        {"role": "system", "content": "Employ concepts as poles on an axis to find their perpendicular partners in a tessellating graph function for exploring concept spaces."},
        {"role": "user", "content": prompt}
      ],
      temperature=0.17,
      )
    print(completion)

    return [choice.text.strip() for choice in completion.choices]

def Groq_call(prompt):
  chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Employ concepts as points along a line to find their perpendicular partners in a tessellating graph function for exploring concept spaces."},
            "role": "user",
            "content": prompt,
        }
    ],
    model="mixtral-8x7b-32768",
  )
  print(chat_completion.choices[0].message.content)
  return chat_completion.choices[0].message.content




# Recursive function to build the concept graph
def build_concept_graph(concept1, concept2, depth=0, max_depth=3):
    # Initialize an empty graph using NetworkX
    graph = nx.Graph()

    # Add seed concepts as nodes to the graph
    graph.add_node(concept1)
    graph.add_node(concept2)

    # Base case: if maximum depth is reached, return the graph
    if depth >= max_depth:
        return graph

    # Query LLM for related concepts perpendicular to the current axis
    related_concepts_list = prompt_lm(f"Considering concepts '{concept1}' and '{concept2}' as 2 points on a line between their meanings, what concepts are most perpendicular to this axis? reply concisely as possible with a short concept subject heading or title.")

    # Add retrieved concepts as nodes to the graph
    for related_concept in related_concepts_list:
        if related_concept not in graph.nodes:
            graph.add_node(related_concept)

        # Add edges between current concepts and retrieved concepts
        graph.add_edge(concept1, related_concept)
        graph.add_edge(concept2, related_concept)

        # Recursively call the function with the new perpendicular concept
        child_graph = build_concept_graph(concept1, related_concept, depth=depth+1, max_depth=max_depth)
        child_graph = build_concept_graph(concept2, related_concept, depth=depth+1, max_depth=max_depth)

        # Merge the child graphs into the main graph
        graph = nx.compose(graph, child_graph)

    return graph

# Main program flow
if __name__ == "__main__":
    # Specify your seed concepts
    concept1 = "Cat"
    concept2 = "Dog"

    # Set the maximum depth for the recursive exploration
    max_depth = 3

    concept_graph = build_concept_graph(concept1, concept2, max_depth=max_depth)

    # Analyze or visualize the concept graph (using NetworkX or other libraries)
    # output graph to text file via write()
    nx.write_dot(concept_graph, "concept_graph.dot")
    # now output as plain text using file open() and write()
    with open("concept_graph.txt", "w") as f:
        f.write(nx.nx_pydot.to_pydot(concept_graph).to_string())
